In [1]:
import json
from pprint import pprint
import re
from pyspark.sql import SparkSession
import nltk.data

spark  = SparkSession.builder.master('local').appName('sou').getOrCreate()
sc = spark.sparkContext
lotr = spark.sparkContext.wholeTextFiles('books').values()

In [2]:
from g2p_seq2seq.g2p import G2PModel
import tensorflow as tf
import sys
import os

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = lotr.flatMap(lambda x: tokenizer.tokenize(x))

stdout_backup = sys.stdout
#sys.stdout = open(os.devnull, "w")

g2p_model = G2PModel('g2p-seq2seq-cmudict')
g2p_model.load_decode_model()

Loading vocabularies from g2p-seq2seq-cmudict
Creating 2 layers of 512 units.
Reading model parameters from g2p-seq2seq-cmudict


In [3]:
import inflect
from operator import add 
from StringIO import StringIO
import string

iengine = inflect.engine()
#sys.stdout = open(os.devnull, "w")

def is_number(s):
    try:
        f = int(s)
        return True
    except:
        return False

alpha = 0.05
    
def sentence_to_phoneme(sentence):
    words = []
    for word in re.split('\W+', sentence):
        if word == '':
            continue
        
        if is_number(word):
            word = iengine.number_to_words(int(word))
        word = word.lower().replace(",", "")
        words += word.split("-")
        
    return words

#phonemes = sc.parallelize(map(sentence_to_phoneme, sentences.collect()))
#sys.stdout = stdout_backup
phonemes = sentences.map(sentence_to_phoneme)
phonemes.first()

#sentence_to_phoneme(sentences.first())

[u'the', u'lord', u'of', u'the', u'rings', u'by', u'j', u'r', u'r']

In [4]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [6]:
#sys.stdout = open(os.devnull, "w")
f = open('phonemes.txt', "w+")
l = []
for sentence in log_progress(phonemes.collect(), every=1):
    ph = g2p_model.decode(sentence, StringIO())
    f.write(" ".join(ph) + "\n")
    l.append(ph)
    
    
phonemes = sc.parallelize(l)
phonemes.count()

Symbols '5','0' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',

Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in v

Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' 

Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols '1','8','3' are not in vocabulary
Symbols ' ' are not in vocabulary
Symbols ' ' are not in vocabulary
Symbols ' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ' are not in voc

Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' 

...

Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in voc

Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in voc

Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in voc

Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary
Symbols ' ',' ',' ' are not in vocabulary


31351

In [18]:
total = phonemes.count()
freq = phonemes.map(lambda x: tuple([tuple(s.split()) for s in x])).cache().reduce(add)#.reduce(add).map(lambda x: (x, 1)).reduceByKey(add)
freq.first()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 25, localhost, executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1435)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1423)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1422)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1422)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFail

In [ ]:
def phoneme_agg(*list_of_words):
    d = {}
    for word in list_of_words:
        d[tuple(word)] = d.get(tuple(word), 0) + 1
        """for ph in word:
            if len(ph) < 3:
                d[ph] = d.get(ph, 0) + 1"""
            
    return d

freq = sc.textFile('phoneme_output.txt', 10).map(lambda x: x.split()[1:])
freq_d = freq.reduce(phoneme_agg)

In [ ]:
freq_d.keys()